# Generate Word Frequency in Audit Reports

## Steps Taken

1. Download the folder with audit reports from OneDrive
2. Iterate through the folder and get the folders with each country's reports
3. For each country's folder, get the country's reports
4. Lemmatize the words in each report
5. Filter the words to remove noise
6. Get the frequency distribution of each text
7. Enter the frequency distribution into a pandas dataframe
8. Merge each country's frequency distributions into one dataframe with the words as rows and report name as column
9. Enter each country as a spreadsheet in an excel workbook
10. Delete the downloaded documents

In [1]:
# link to audit_reports; publicly accessible
audit_reports_link = "https://stir-my.sharepoint.com/:f:/g/personal/fkc3_stir_ac_uk/Esgp-VMQyzBClY5vpTP9TsYBTCb16iA3NvelLEJM53VEgQ?e=7ejaR3"

### 2. Iterate through the folder and create a dictionary to match each country to files within it

In [2]:
import os
from pdfminer.high_level import extract_text
import nltk
from nltk.corpus import stopwords
import pandas as pd

In [3]:
audit_reports_file_path = "./audit_reports"

In [17]:
country_audit_report_dict = {}

for root, dirs, files in os.walk(audit_reports_file_path):
    if (dirs): # ignore the country folders as children
        continue
    country_audit_report_dict[root] = sorted(files);

### 3. For each country's folder, get the country's reports

In [5]:
country_texts_dict = {}

for country_folder_path, filenames in country_audit_report_dict.items():
    for filename in filenames:
        print(f'Doing {country_folder_path}/{filename}')
        try:
            text = extract_text(f'{country_folder_path}/{filename}').lower()
        
            if text and country_folder_path in country_texts_dict:
                country_texts_dict[country_folder_path][filename] = text;
            elif text and country_folder_path not in country_texts_dict:
                country_texts_dict[country_folder_path] = {filename: text}
        except:
            print(f'{filename} has a problem')


### 4. Lemmatize the words in each report

In [6]:
lemmatizer = nltk.WordNetLemmatizer()

for country_path in country_texts_dict.keys():
    for report_name, report_text in country_texts_dict[country_path].items():
        if report_text:
            words = nltk.word_tokenize(report_text.lower())            
            words = [lemmatizer.lemmatize(word) for word in words]

            country_texts_dict[country_path][report_name] = words

### 5. Filter the words to remove noise

In [7]:
# remove words made up numbers, symbols, and stopwords
def is_ideal_word(word):
    if word.isalpha() and word not in stopwords.words('english'):
        return True
    return False

    
for country_path in country_texts_dict.keys():
    for report_name, words in country_texts_dict[country_path].items():
        words = list(filter(is_ideal_word, words))
        country_texts_dict[country_path][report_name] = words

### 6. Get the frequency distribution of each text

In [8]:
country_report_freq_dist_dict = {}

for country_path in country_texts_dict.keys():
    for report_name, words in country_texts_dict[country_path].items():
        fdist = nltk.FreqDist(words)

        if country_path in country_report_freq_dist_dict:
            country_report_freq_dist_dict[country_path][report_name] = fdist
        else:
            country_report_freq_dist_dict[country_path] = {report_name: fdist}

### 7. Enter the frequency distributions into a pandas dataframe

In [9]:
country_report_df_dict = {}

for country_path in country_report_freq_dist_dict.keys():
    for report_name, freq_dist in country_report_freq_dist_dict[country_path].items():
        df = pd.DataFrame.from_dict(dict(freq_dist), orient='index', columns=[report_name])

        if country_path in country_report_df_dict:
            country_report_df_dict[country_path][report_name] = df
        else:
            country_report_df_dict[country_path] = {report_name: df}

### 8. Merge each country's frequency distributions into one dataframe with the words as rows and report name as column

In [10]:
country_df_dict = {}

for country_path in country_report_df_dict.keys():
    df = pd.DataFrame()
    for report_df in country_report_df_dict[country_path].values():
        if df.empty:
            df = report_df
        else:
            df = pd.merge(df, report_df, 'outer',left_index=True, right_index=True)
    country_df_dict[country_path] = df

In [11]:
country_df_dict['./audit_reports/Nigeria'][:100]

KeyError: './audit_reports/Nigeria'

In [ ]:
country_df_dict['./audit_reports/Malawi'].loc['transparency']

### 9. Enter each country as a spreadsheet in an excel workbook

In [ ]:
with pd.ExcelWriter('word-frequency.xlsx') as writer:
    for country_path, df in country_df_dict.items():
        df.to_excel(writer, sheet_name=f'{country_path.removeprefix("./audit_reports/")}')

In [ ]:
d = {'report': 42, 'auditor': 143, 'general': 185}
p = pd.DataFrame.from_dict(d, orient='index', columns=['2022'])
p

In [ ]:
zam_2015 = country_report_df_dict['./audit_reports/Zambia']['ZAM_GenGov_2015.pdf']
zam_2015